In [1]:
import os

os.environ["SEED"] = "0"
import numpy as np
import pandas as pd
from ConfigSpace import Configuration

from src.aac.AAC import AAC
from src.configuration_space.LKH import CONFIGURATION_SPACE
from src.constant import DATA_DIR, SEED
from src.database import DB
from src.database.queries import *
from src.instance.Instance import Instance
from src.instance.InstanceList import InstanceList
from src.instance.TSP_Instance import TSP_Instance, TSP_train_test_from_index_file
from src.solver.Portfolio import Portfolio
from src.solver.TSP_LKH_Solver import TSP_LKH_Solver
from src.aac.SurrogateEstimator import Estimator1, SurrogateEstimator

In [2]:
train_instances, test_instances = TSP_train_test_from_index_file(
    filepath=DATA_DIR / "TSP" / "CEPS_benchmark" / "index.json",
    train_size=3,
)
train_instances

InstanceList(size=3)[TSP_Instance(filepath=TSP/CEPS_benchmark/explosion/21.tsp), TSP_Instance(filepath=TSP/CEPS_benchmark/rotation/34.tsp), TSP_Instance(filepath=TSP/CEPS_benchmark/compression/09.tsp)]

In [3]:
portfolio = Portfolio.from_solver_class(TSP_LKH_Solver, size=1)
portfolio

Portfolio(size=1)[Solver(id=52980316141715150)]

In [4]:
aac = AAC(
    portfolio=portfolio,
    instance_list=train_instances,
    prefix="config",
    t_c=180,
    calculate_features=False,
    estimator=None,
)
db = DB()

for _ in aac.configure_iter():
    for idx, instance in enumerate(train_instances):
        
        number_of_no_timeouts = get_number_of_no_timeouts(db, instance.id(), TSP_LKH_Solver.MAX_COST)
        if number_of_no_timeouts >= 0:
            new_instance, time = instance.mutate()
            train_instances[idx] = new_instance
            aac._update_configuration_time(time)
        aac.update(instance_list=train_instances)
    train_instances.log()

[2024-12-29 00:28:07] DEBUG     AAC(prefix=config, iter=1, configuration_time=[180.00])
[2024-12-29 00:28:07] DEBUG     Portfolio.evaluate(config;aac_iter=1)
[2024-12-29 00:28:07] DEBUG     Portfolio(size=1)[Solver(id=52980316141715150)]
[2024-12-29 00:28:07] DEBUG     solve(prefix=config;aac_iter=1, solver=Solver(id=52980316141715150), instance=TSP_Instance(filepath=TSP/CEPS_benchmark/explosion/21.tsp))
[2024-12-29 00:28:07] DEBUG     solve(prefix=config;aac_iter=1, solver=Solver(id=52980316141715150), instance=TSP_Instance(filepath=TSP/CEPS_benchmark/rotation/34.tsp))
[2024-12-29 00:28:07] DEBUG     solve(prefix=config;aac_iter=1, solver=Solver(id=52980316141715150), instance=TSP_Instance(filepath=TSP/CEPS_benchmark/compression/09.tsp))
[2024-12-29 00:28:19] DEBUG     Solver.Result(prefix=config;aac_iter=1, solver=Solver(id=52980316141715150), instance=TSP_Instance(filepath=TSP/CEPS_benchmark/explosion/21.tsp), cost=100.00, time=10.00, cached=False, surrogate=False, error=False)
[202

In [2]:
# db = DB()
db = DB("database/run-plain_924039.db")

In [3]:
pd.read_sql_query("SELECT * FROM instances", db._conn)

,id,filepath,optimum,angle_min,angle_median,angle_mean,angle_max,angle_sd,angle_span,angle_coef_of_var,...,bc_no1s_min,bc_no1s_q25,bc_no1s_q50,bc_no1s_q75,bc_no1s_max,bc_p1s,bc_pn1s,bcTime,acc,acfTime
0,855157651668393360,TSP/CEPS_benchmark/explosion/21.tsp,14928292.0,0.006290,1.484946,1.522267,3.139028,0.916031,3.132738,0.601754,...,0.125000,0.500000,0.5,0.500000,0.875000,0.90125,0.20000,0.20,389.435780,0.68
1,1353678304570487518,TSP/CEPS_benchmark/rotation/34.tsp,15486460.0,0.000684,1.495912,1.523094,3.138384,0.912371,3.137700,0.599025,...,0.111111,0.500000,0.5,0.500000,0.888889,0.87750,0.24750,0.13,408.796973,0.72
2,810870798045289410,TSP/CEPS_benchmark/compression/09.tsp,16003060.0,0.000000,1.417332,1.483901,3.141593,1.026220,3.141593,0.691569,...,0.166667,0.500000,0.5,0.500000,0.833333,0.87125,0.26500,0.10,397.532639,0.61
3,1306299629087479721,TSP/CEPS_benchmark/implosion/34.tsp,19968838.0,0.000395,1.483846,1.518058,3.138849,0.916994,3.138454,0.604057,...,0.125000,0.333333,0.5,0.666667,0.875000,0.86500,0.27375,0.08,419.815336,0.42
4,410041616729589825,TSP/CEPS_benchmark/linearprojection/25.tsp,13578491.0,0.009872,1.625140,1.570276,3.131472,0.931936,3.121600,0.593485,...,0.250000,0.500000,0.5,0.500000,0.750000,0.87500,0.25500,0.10,384.777585,0.57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,707351994282137341,TSP/CEPS_benchmark/cluster/19.tsp,19287084.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,540324668286432217,TSP/CEPS_benchmark/uniform_portgen/27.tsp,20825913.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,1765803814609198146,TSP/CEPS_benchmark/grid/00.tsp,21258773.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,1378924496111515287,TSP/CEPS_benchmark/explosion/32.tsp,12927140.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
pd.read_sql_query("SELECT * FROM solvers", db._conn)

,id,ASCENT_CANDIDATES,BACKBONE_TRIALS,BACKTRACKING,CANDIDATE_SET_TYPE,EXTRA_CANDIDATES,EXTRA_CANDIDATE_SET_TYPE,GAIN23,GAIN_CRITERION,INITIAL_STEP_SIZE,...,MAX_BREADTH,MAX_CANDIDATES,MOVE_TYPE,PATCHING_A,PATCHING_C,POPULATION_SIZE,RESTRICTED_SEARCH,SUBGRADIENT,SUBSEQUENT_MOVE_TYPE,SUBSEQUENT_PATCHING
0,52980316141715150,0.60,1.0,0.0,0.0,0.7,0.0,1.0,0.0,0.00,...,0.444643,1.000000,0.25,0.6,0.2,0.061224,0.0,0.0,5.0,1.0
1,287011037061967493,0.75,0.0,0.0,0.0,0.3,0.0,1.0,0.0,0.00,...,0.592453,0.777778,0.75,0.8,0.2,0.836735,1.0,1.0,3.0,1.0
2,176228367859327934,0.85,1.0,0.0,0.0,0.4,0.0,0.0,1.0,0.50,...,0.354664,0.333333,0.25,0.8,0.4,0.408163,1.0,1.0,2.0,0.0
3,2041120758543581287,0.80,1.0,0.0,2.0,0.7,0.0,1.0,0.0,0.25,...,0.365942,1.000000,0.25,0.6,0.8,0.051020,0.0,1.0,5.0,0.0
4,1056464823932360013,0.65,1.0,1.0,2.0,0.0,0.0,0.0,1.0,1.00,...,0.322513,0.666667,1.00,0.8,0.8,0.112245,0.0,1.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,2117566868330776600,0.95,1.0,0.0,1.0,0.5,0.0,1.0,1.0,0.50,...,0.745170,0.555556,0.75,0.6,0.2,0.775510,1.0,1.0,2.0,1.0
185,959378174096938100,0.45,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.50,...,0.916505,0.444444,0.00,0.0,0.6,0.857143,1.0,0.0,3.0,0.0
186,554296385581430461,0.40,0.0,0.0,0.0,0.3,0.0,0.0,0.0,1.00,...,0.546980,0.888889,0.00,0.0,0.0,0.846939,1.0,0.0,3.0,0.0
187,1132140047627215859,0.95,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.50,...,0.477661,0.222222,0.00,0.4,0.4,0.387755,1.0,0.0,3.0,1.0


In [5]:
results = pd.read_sql_query("SELECT * FROM results", db._conn)
results

,id,prefix,solver_id,instance_id,cost,time,cached,surrogate,error
0,config;aac_iter=1_52980316141715150_8551576516...,config;aac_iter=1,52980316141715150,855157651668393360,100.00,22.182200,0,0,0
1,config;aac_iter=1_52980316141715150_1353678304...,config;aac_iter=1,52980316141715150,1353678304570487518,100.00,16.941617,0,0,0
2,config;aac_iter=1_52980316141715150_1306299629...,config;aac_iter=1,52980316141715150,1306299629087479721,100.00,15.190077,0,0,0
3,config;aac_iter=1_52980316141715150_8108707980...,config;aac_iter=1,52980316141715150,810870798045289410,100.00,15.999449,0,0,0
4,config;aac_iter=1_52980316141715150_4100416167...,config;aac_iter=1,52980316141715150,410041616729589825,100.00,17.040042,0,0,0
...,...,...,...,...,...,...,...,...,...
1225,test3_906442550283152180_1991714966268353646,test3,906442550283152180,1991714966268353646,3.17,3.170000,0,0,0
1226,test3_906442550283152180_366756074013316631,test3,906442550283152180,366756074013316631,0.57,0.570000,0,0,0
1227,test3_906442550283152180_540324668286432217,test3,906442550283152180,540324668286432217,5.68,5.680000,0,0,0
1228,test3_906442550283152180_1667136710979054795,test3,906442550283152180,1667136710979054795,8.35,8.350000,0,0,0


In [6]:
_ = results.loc[results["prefix"].str.startswith("config")].copy()
_["no_timeout"] = _["time"] < 10
_.groupby("instance_id")["no_timeout"].value_counts()

instance_id          no_timeout
1306299629087479721  False         108
                     True           81
1353678304570487518  False         112
                     True           77
410041616729589825   False         105
                     True           84
810870798045289410   False         121
                     True           68
855157651668393360   False         157
                     True           32
Name: no_timeout, dtype: int64

In [12]:
get_number_of_no_timeouts(db, "855157651668393360", TSP_LKH_Solver.MAX_COST)

32

In [ ]:
pd.read_sql_query("SELECT * FROM evaluations", db._conn)